## Scenario 1: A single data scientist participating in an ML competition

MLflow setup:
* Tracking server: no
* Backend store: local filesystem
* Artifacts store: local filesystem

The experiments can be explored locally by launching the MLflow UI.

In [1]:
import mlflow

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///e:/Projects/Git/mlops_selflearn/02-experiment-tracking/running-mlflow-examples/mlruns'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///e:/Projects/Git/mlops_selflearn/02-experiment-tracking/running-mlflow-examples/mlruns/0', creation_time=1685204596837, experiment_id='0', last_update_time=1685204596837, lifecycle_stage='active', name='Default', tags={}>]

### Creating an experiment and logging a new run

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2023/05/28 00:23:17 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'file:///e:/Projects/Git/mlops_selflearn/02-experiment-tracking/running-mlflow-examples/mlruns/267092520342137636/0625e042f8ba4dd8934b9579c0ca62d6/artifacts'


d:\Softwares\anaconda3\envs\exp-tracking-env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///e:/Projects/Git/mlops_selflearn/02-experiment-tracking/running-mlflow-examples/mlruns/267092520342137636', creation_time=1685204597964, experiment_id='267092520342137636', last_update_time=1685204597964, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///e:/Projects/Git/mlops_selflearn/02-experiment-tracking/running-mlflow-examples/mlruns/0', creation_time=1685204596837, experiment_id='0', last_update_time=1685204596837, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [7]:
from mlflow.exceptions import MlflowException

# Note: registering model is possible on local filesystem for mLFlow > 2.0, see code cell below
# try:
client.search_registered_models()
# except MlflowException:
#     print("It's not possible to access the model registry :(")

In [18]:
run_id = client.search_runs(experiment_ids='267092520342137636')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2023/05/28 00:25:58 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1685204758732, current_stage='None', description=None, last_updated_timestamp=1685204758732, name='iris-classifier', run_id='0625e042f8ba4dd8934b9579c0ca62d6', run_link=None, source='file:///e:/Projects/Git/mlops_selflearn/02-experiment-tracking/running-mlflow-examples/mlruns/267092520342137636/0625e042f8ba4dd8934b9579c0ca62d6/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=1>